In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np
from typing import List, Callable, Dict
from realgam.quantlib import general_utils as gu
from realgam.quantlib.engineer.op_engineer import OpEngineer, GroupOpEngineer
from realgam.quantlib.engineer.ta_engineer import TalibEngineer, GroupTalibEngineer
from realgam.quantlib.engineer.alpha_engineer import AlphaEngineer
from realgam.quantlib.engineer.interface import BaseEngineer, GroupBaseEngineer
import datetime
import os
import time
PROJECT_PATH = os.getenv('QuantSystemMVP')
DATA_PATH = f'{PROJECT_PATH}/Data/stock_hist.obj'

In [2]:
stocks_df, stocks_extended_df, available_tickers = gu.load_file(DATA_PATH)
stacked_hist = stocks_df.copy()

In [3]:
goe0 = GroupOpEngineer(stacked_hist, 'ticker')

In [4]:
goe0.ts_rank('volume', 10, inplace=True)

KeyboardInterrupt: 

In [4]:
goe = GroupOpEngineer(tripA, 'ticker')

ValueError: Length of values (1) does not match length of index (5369)

In [ ]:
goe0.ts_pctrank('volume', 10, inplace=True)

In [49]:
def rank_last_value(x, shift= 2, pct=False):
    """
    One step of the procedure to get the rolling rank of the last value
    according to the previous window of values.

    Use the bottlneck routine below instead. It is much faster. Keep this here for testing.
    Use with .rolling(window=100).apply(rank_last_value, raw=True) for example
    """
    args = np.argsort(x)
    rank = np.argwhere(args == (x.shape[0] - 1))[0][0]
    if pct:
        return (rank + 1)/ (x.shape[0] + shift)
    return rank

In [9]:
tickers = stacked_hist.ticker.unique()[:1000]
tripA = stacked_hist[stacked_hist.ticker.isin(tickers)]
A = stacked_hist[stacked_hist.ticker.isin(['A'])]

In [21]:
goe = GroupOpEngineer(tripA, 'ticker')
goe.ts_argmin('volume', 6)

ticker  date      
A       2012-01-03    NaN
        2012-01-04    NaN
        2012-01-05    NaN
        2012-01-06    NaN
        2012-01-09    NaN
                     ... 
AXH     2022-03-10    2.0
        2022-03-11    1.0
        2022-03-14    5.0
        2022-03-15    4.0
        2022-03-16    3.0
Name: volume, Length: 1290427, dtype: float64

In [18]:
goe.df

,ticker,open,high,low,close,openadj,highadj,lowadj,closeadj,volume,cs_rank_volume
date,,,,,,,,,,,
2012-01-03,A,35.84,36.990,35.740,36.48,23.651649,24.410561,23.585657,24.074,2973100.0,443.0
2012-01-04,A,36.15,36.360,35.590,36.19,23.855604,23.994184,23.486056,23.882,3327500.0,448.0
2012-01-05,A,35.88,37.320,35.720,37.00,23.677891,24.628174,23.572304,24.417,4894600.0,460.0
2012-01-06,A,37.09,37.555,36.805,37.40,24.476425,24.783288,24.288348,24.681,3370100.0,447.0
2012-01-09,A,37.62,38.420,37.472,38.38,24.826455,25.354397,24.728786,25.328,3168500.0,449.0
...,...,...,...,...,...,...,...,...,...,...,...
2022-03-10,AXH,9.98,9.980,9.980,9.98,9.980000,9.980000,9.980000,9.980,3374.0,102.0
2022-03-11,AXH,9.98,9.980,9.980,9.98,9.980000,9.980000,9.980000,9.980,90.0,28.0
2022-03-14,AXH,9.98,9.980,9.980,9.98,9.980000,9.980000,9.980000,9.980,446131.0,396.0


In [8]:
dummy = tripA.groupby('date').volume.apply(np.argsort)
dummy1 = (dummy + 1) / (dummy.shape[0] + 2)

In [9]:
dummy1

date
2012-01-03    0.000049
2012-01-03    0.000335
2012-01-03    0.000245
2012-01-03    0.000409
2012-01-03    0.000221
                ...   
2022-03-16    0.000294
2022-03-16    0.000196
2022-03-16    0.000245
2022-03-16    0.000074
2022-03-16    0.000123
Name: volume, Length: 122318, dtype: float64

In [24]:
tripA.groupby('ticker')[['volume','lowadj']].rolling(10).cov().iloc[:,-1].reset_index().drop_duplicates(['ticker','date'], keep='first').iloc[:, -1]

0                 NaN
2                 NaN
4                 NaN
6                 NaN
8                 NaN
             ...     
244626   -1913.992000
244628   -2050.041278
244630    -106.669944
244632       0.580156
244634    -702.126611
Name: lowadj, Length: 122318, dtype: float64

In [23]:
tripA.groupby('ticker')[['volume','lowadj']].rolling(10).cov()

volume      lowadj
ticker date                                       
A      2012-01-03 volume           NaN         NaN
                  lowadj           NaN         NaN
       2012-01-04 volume           NaN         NaN
                  lowadj           NaN         NaN
       2012-01-05 volume           NaN         NaN
...                                ...         ...
ACER   2022-03-14 lowadj -1.066699e+02    0.001094
       2022-03-15 volume  3.722937e+08    0.580156
                  lowadj  5.801556e-01    0.001407
       2022-03-16 volume  2.347120e+09 -702.126611
                  lowadj -7.021266e+02    0.001304

[244636 rows x 2 columns]

In [42]:
goe0 = GroupOpEngineer(A, 'ticker')

In [7]:
goe0.ts_rank('volume', 3, inplace=True)

In [11]:
dummy = A.iloc[:, 1:]

In [44]:
tripA1 = tripA[['ticker','volume']]

In [45]:
ofe0 = GroupOpEngineer(tripA1, 'ticker')

In [53]:
s_time_chunk = time.time()
dummy = tripA1.groupby('ticker').volume.rolling(10).apply(rank_last_value, raw=True)
e_time_chunk = time.time()
print(f"Total time: {e_time_chunk - s_time_chunk} sec")


Total time: 5.396774768829346 sec


In [55]:
tripA['rank'] = dummy.values

In [56]:
tripA

,ticker,open,high,low,close,openadj,highadj,lowadj,closeadj,volume,rank
date,,,,,,,,,,,
2012-01-03,A,35.84,36.990,35.740,36.48,23.651649,24.410561,23.585657,24.074,2973100.0,NaN
2012-01-04,A,36.15,36.360,35.590,36.19,23.855604,23.994184,23.486056,23.882,3327500.0,NaN
2012-01-05,A,35.88,37.320,35.720,37.00,23.677891,24.628174,23.572304,24.417,4894600.0,NaN
2012-01-06,A,37.09,37.555,36.805,37.40,24.476425,24.783288,24.288348,24.681,3370100.0,NaN
2012-01-09,A,37.62,38.420,37.472,38.38,24.826455,25.354397,24.728786,25.328,3168500.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2017-10-02,AMFW,6.66,6.680,6.630,6.66,6.660000,6.680000,6.630000,6.660,12407.0,0.0
2017-10-03,AMFW,6.64,6.820,6.580,6.67,6.640000,6.820000,6.580000,6.670,1651903.0,9.0
2017-10-04,AMFW,6.72,6.920,6.680,6.87,6.720000,6.920000,6.680000,6.870,415055.0,8.0


In [52]:
s_time_chunk = time.time()
dummy = tripA1.groupby('ticker').volume.rolling(10).apply(lambda x: pd.Series(x).rank().iloc[-1])
e_time_chunk = time.time()
print(f"Total time: {e_time_chunk - s_time_chunk} sec")


KeyboardInterrupt: 

In [46]:
s_time_chunk = time.time()
ofe0.ts_rank('volume', 10, inplace=True)
e_time_chunk = time.time()
print(f"Total time: {e_time_chunk - s_time_chunk} sec")

Total time: 80.64372301101685 sec


In [47]:
tripA2 = tripA[['ticker', 'volume']].reset_index()
tripA2 = tripA2.pivot(index='date', columns='ticker', values= 'volume')

In [48]:
s_time_chunk = time.time()
dummy = tripA2.rolling(10).apply(lambda x: pd.Series(x).rank().iloc[-1], engine='cython', raw=True)
e_time_chunk = time.time()
print(f"Total time: {e_time_chunk - s_time_chunk} sec")

Total time: 80.60853600502014 sec


In [23]:
tripA2 = tripA[['ticker', 'volume']].reset_index()

In [25]:
tripA2 = tripA2.pivot(index='date', columns='ticker', values= 'volume')

In [26]:
tripA2

ticker,A,AA,AAP
date,,,
2012-01-03,2973100.0,NaN,786600.0
2012-01-04,3327500.0,NaN,1412800.0
2012-01-05,4894600.0,NaN,596900.0
2012-01-06,3370100.0,NaN,1346600.0
2012-01-09,3168500.0,NaN,686100.0
...,...,...,...
2022-03-10,1933127.0,10227270.0,613411.0
2022-03-11,1815743.0,5687948.0,527477.0
2022-03-14,1737099.0,7906373.0,1229028.0


In [4]:
s_time_chunk = time.time()
afe0.alpha2(2, 6, inplace=True)
e_time_chunk = time.time()

print(f"Total time: {e_time_chunk - s_time_chunk} sec")

/Users/punlee/miniconda3/envs/quant/lib/python3.9/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Total time: 4843.606370210648 sec
